In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
from graph_utils import graph_gen
from algorithms import IPLUX

# Set up the logger to print info messages for understandability.
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

# from algorithms import B_DPP, Falsone, C_SP_SG, DPD_TV

num_node = 20
num_edge = 40

graph_gen(num_node, num_edge, B=1)

net_dir = f'data/graph/N{num_node}E{num_edge}'
network = np.load(f'{net_dir}/subgraph_W.npy')
print(network.shape)
# print(network)

Graph with 20 nodes and 40 edges
(20, 20)


C:\Users\xtq\AppData\Roaming\Python\Python311\site-packages\networkx\drawing\nx_pylab.py:315: UserWarning: 

The arrowsize keyword argument is not applicable when drawing edges
with LineCollection.

To make this warning go away, either specify `arrows=True` to
force FancyArrowPatches or use the default values.
Note that using FancyArrowPatches may be slow for large graphs.

  draw_networkx_edges(G, pos, arrows=arrows, **edge_kwds)


In [30]:
from problem import ElecVehCharg

prob = ElecVehCharg(num_node)
prob.gen()
# prob.gen(no_ineq=True)
prob.load()


generating a ElecVehCharg problem, N=20
x* [ 1.00e+00  5.42e-10  1.00e+00  2.78e-08  4.06e-01  4.95e-10  1.97e-10
  1.29e-10  2.46e-10  4.81e-10  4.55e-10  2.60e-10  3.64e-10  3.98e-01
  1.00e+00  1.00e+00  2.11e-10  4.82e-02  1.39e-01  1.00e+00], f* 0.3865154318238943
generated problem saved in data/problem/ElecVehCharg/N20
loading a ElecVehCharg problem, N=20
problem loaded:
c (20,)
d (20,)
b (1,)
x_star (20,)
opt_val 0.3865154318238943


In [31]:
MAX_ITER = 500

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

alg = IPLUX(prob, network, alpha=1, rho=10, verbose=0)
# alg = IPLUX(prob, network, alpha=20, rho=0.05, no_ineq=True)

# logging.info(f'{labels[algo_idx]} is running:')

np.set_printoptions(formatter={'float':lambda x: f' {x:.2e}' if x>0 else f'{x:.2e}'})

for t in range(MAX_ITER):
    alg.step()
    obj_err, cons_vio = alg.compute_metrics()
    obj_err_avg, cons_vio_avg = alg.compute_metrics(avg=True)
    
    if alg.iter_num % 100 == 0:
        logging.info(f'iter {alg.iter_num}, obj err: {obj_err:.2e}, cons vio: {cons_vio:.2e}')
# obj_err_list.append(obj_err_log)
# cons_val_list.append(cons_vio_log)






self.prob minimize param3586[0] @ var3585[0] + 0.5 @ QuadForm(var3585 + -param3587, [[ 1.00e+00]]) + param3588[0] @ -log(var3585[0] + 1.0)
subject to 0.0 <= var3585
           var3585 <= 1.0
reset
saved
running time for IPLUX = 4.90
iter 100, obj err: 8.31e-02, cons vio: 7.76e-11
saved
running time for IPLUX = 9.54
iter 200, obj err: 1.39e-01, cons vio: 5.29e-11
saved
running time for IPLUX = 14.29
iter 300, obj err: 1.40e-01, cons vio: 5.31e-11
saved
running time for IPLUX = 18.93
iter 400, obj err: 1.39e-01, cons vio: 5.31e-11
saved
running time for IPLUX = 23.49
iter 500, obj err: 1.39e-01, cons vio: 5.32e-11


In [32]:
# =========================================================================== |
# ---------------------------------- Plot ----------------------------------- |
# =========================================================================== |


from plot_utils import MyFigure

# ================================ objective error============================
obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel=r'$\sum_{i=1}^N f_i(\bar{x}_{i,t}) - \sum_{i=1}^N f_i(x_i^\star)$',
                            yscale='log')
obj_err_figure.add_line("IPLUX", alg.obj_err_log)
obj_err_figure.add_line("IPLUX avg", alg.obj_err_avg_log)
obj_err_figure.paint(MAX_ITER=MAX_ITER)


# ================================= constraint violation ====================
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            # ylabel=r'$\left\vert\sum_{i=1}^N g_i(\bar{x}_{i,t})\right\vert$',
                            ylabel='constraint violation',
                            yscale='log')
cons_vio_figure.add_line("IPLUX", alg.cons_vio_log)
cons_vio_figure.add_line("IPLUX avg", alg.cons_vio_avg_log)
cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)



# ============================= trace in x1 x1 plane =========================
import matplotlib.pyplot as plt
import matplotlib.cm as cm

fig, ax = plt.subplots()     # Create a figure containing a single Axes.

delta = 0.05
x = np.arange(0.0, 1, delta)
y = np.arange(0.0, 1, delta)
X, Y = np.meshgrid(x, y)
F = X*alg.c[0] + Y*alg.c[1]
G = -alg.d[0]*np.log(1+X) - alg.d[1]*np.log(1+Y) + alg.b[0]*2/num_node

im = ax.imshow(F, interpolation='bilinear', origin='lower',
            cmap=cm.gray, extent=(0, 1, 0, 1))
CS_F = ax.contour(X, Y, F, 5)
ax.clabel(CS_F, fontsize=50)

# if there are only two nodes, we can polt the constraint
if num_node == 2:
    CS_G = ax.contour(X, Y, G, colors='k')
    ax.clabel(CS_G, fontsize=20)

x_log = np.array(alg.x_log)
x_avg_log = np.array(alg.x_avg_log)
ax.scatter(alg.x_star[0], alg.x_star[1], s=5000, c='r', marker='X', label='optimal')
ax.scatter(x_log[-1,0], x_log[-1,1], s=500, c='y', marker='^', label='last')
ax.scatter(x_log[:,0], x_log[:,1], label='iter', s=100, c='b', marker='^')  # Plot some data on the Axes.
ax.scatter(x_avg_log[:,0], x_avg_log[:,1], label='iter avg', s=100, marker='^')

ax.set_title('x position')
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.legend()
fig.savefig('x.png')
plt.close()


In [7]:
print(alg.W)
print(alg.H)

[[ 6.50e-01 0.00e+00 0.00e+00 0.00e+00  1.00e-01 0.00e+00  8.33e-02
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00  8.33e-02 0.00e+00
  0.00e+00 0.00e+00 0.00e+00  8.33e-02 0.00e+00 0.00e+00]
 [0.00e+00  8.19e-01 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00  1.25e-01 0.00e+00 0.00e+00 0.00e+00
   5.56e-02 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00  7.45e-01 0.00e+00  1.00e-01 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00  7.14e-02 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00  8.33e-02]
 [0.00e+00 0.00e+00 0.00e+00  8.19e-01 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
   5.56e-02 0.00e+00 0.00e+00 0.00e+00  1.25e-01 0.00e+00]
 [ 1.00e-01 0.00e+00  1.00e-01 0.00e+00  6.33e-01 0.00e+00  8.33e-02
  0.00e+00 0.00e+00  8.33e-02 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+0